In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [113]:
id_insp = 2
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_319977 where num_registro={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/33130/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
data['grupo_familiar'] = data['grupo_familiar'].title()
canhero = collections.namedtuple("canhero", data.keys())(*data.values())

In [114]:
query = {'query': 'select * from dataset_319976 where diagnostico_to_obs=\'{id_ref}\' order by categoria'.format(id_ref=canhero.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/33130/sql'
select = amigocloud.get(project, query)
data = select['data']

In [115]:
obs = []
for i in data:
    if i['cumplimiento_seg_1'] == None:
        i['cumplimiento_seg_1'] = ''
    if i['cumplimiento_seg_2'] == None:
        i['cumplimiento_seg_2'] = ''
    if i['cumplimiento_seg_3'] == None:
        i['cumplimiento_seg_3'] = ''
    if i['fecha_seg_1'] == None:
        i['fecha_seg_1'] = ''
    if i['fecha_seg_2'] == None:
        i['fecha_seg_2'] = ''
    if i['fecha_seg_3'] == None:
        i['fecha_seg_3'] = ''
    if i['observaciones'] == None:
        i['observaciones'] = ''
    ob = collections.namedtuple("ob", i.keys())(*i.values())
    obs.append(ob)

In [116]:
doc = DocxTemplate(ruta + '/templates/tpl_rpt_seguimiento_propiedades-bpl.docx')
context = {'canhero':canhero, 'obs':obs}
cod_ca = canhero.nom_ca.split(' / ')[0]
nom_ca = canhero.nom_ca.split(' / ')[1]
nom_file = cod_ca + '_SPR_' + canhero.fecha_registro.replace('/','-') + '_' + nom_ca + '_' + str(canhero.num_registro)
doc.render(context)
doc.save(nom_file + '.docx')